# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [2]:
# Write your code below.
%load_ext dotenv
%dotenv C:/Users/Polar/DSI_modules/production/05_src/.env

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [4]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [5]:
import os
from glob import glob

# Write your code below.
# Load the PRICE_DATA environment variable
price_data_directory='../../05_src/data/prices'
price_data = os.getenv('PRICE_DATA')

# Use glob to find all parquet files in the PRICE_DATA directory
from glob import glob
parquet_files = glob(os.path.join(price_data_directory, '**/*.0.parquet'),recursive=True)
len(parquet_files)


11207

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [7]:
# Write your code below.
# Load the parquet files as a Dask dataframe
import dask.dataframe as dd
ddf = dd.read_parquet(parquet_files)


In [8]:
ddf.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.463039,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.142929,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.652859,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.219204,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.237446,2819626,Health Care,Life Sciences Tools & Services,2000


In [12]:
# Groupby and apply to add Adj_Close_lag
ddf = ddf.groupby('ticker', group_keys=False).apply(lambda x: x.assign(Adj_Close_lag=x['Adj Close'].shift(1)))
print(ddf.columns)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sector',
       'subsector', 'year', 'Adj_Close_lag'],
      dtype='object')


C:\Users\Polar\AppData\Local\Temp\ipykernel_35360\611128023.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf = ddf.groupby('ticker', group_keys=False).apply(lambda x: x.assign(Adj_Close_lag=x['Adj Close'].shift(1)))


In [13]:
# Calculate returns based on Adjusted Close
ddf['returns'] = (ddf['Adj Close'] / ddf['Adj_Close_lag']) - 1 

# Calculate the high-low range
ddf['hi_lo_range'] = ddf['High'] - ddf['Low']

# Assign the result to dd_feat
dd_feat = ddf[['Date', 'Close', 'Adj Close', 'Adj_Close_lag', 'returns',
       'hi_lo_range']]

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [20]:
# Write your code below.
import dask.dataframe as dd
import pandas as pd
import numpy as np


In [21]:
# Example: Create a Dask DataFrame (replace this with your actual data loading)
data = {
    'date': pd.date_range(start='2020-01-01', periods=100),
    'return': np.random.randn(100)  # Replace with your actual return data
}
dask_df = dd.from_pandas(pd.DataFrame(data), npartitions=2)

# Convert the Dask DataFrame to a Pandas DataFrame
pandas_df = dask_df.compute()

# Add a rolling average return calculation with a window of 10 days
pandas_df['rolling_avg_return'] = pandas_df['return'].rolling(window=10).mean()

# Optionally, display the result
print(pandas_df)

         date    return  rolling_avg_return
0  2020-01-01  1.144268                 NaN
1  2020-01-02 -0.143989                 NaN
2  2020-01-03 -2.382736                 NaN
3  2020-01-04  0.100449                 NaN
4  2020-01-05 -0.260722                 NaN
..        ...       ...                 ...
95 2020-04-05 -0.827903           -0.422340
96 2020-04-06 -0.499075           -0.626918
97 2020-04-07  1.192304           -0.541285
98 2020-04-08 -0.493413           -0.413747
99 2020-04-09 -1.336242           -0.571657

[100 rows x 3 columns]


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return? _Yes._
+ Would it have been better to do it in Dask? Why? _Generally better to perform rolling average calcuation in Dask especially for larger sets._

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.